In [1]:
pip install twint

In [2]:
pip install emoji

In [3]:
pip install googletrans

In [0]:
import textblob
import re
import emoji


import matplotlib.pyplot as plt
import pandas as pd

import twint
from googletrans import Translator



In [0]:
#define some function
def percent(part, total):
    return 100*float(part)/float(total)

In [0]:

def clean_emoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [0]:
def remove_link_mention(text):
  pat1 = r'@[A-Za-z0-9]+'
  pat2 = r'https?://[A-Za-z0-9./]+'
  combined_pat = r'|'.join((pat1, pat2))
  string = " " .join([re.sub(combined_pat, '', text) for str in text.split()])
  return string


In [0]:
def remove_punct(text):
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text  = "".join([char for char in text if char not in punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [0]:
def sentimentlysis(string):
  string = remove_link_mention(string)
  string = clean_emoji(string)
  string = remove_punct(string)
  #string = Translator().translate(str(string), dest='en') karena api limit pake bahasa inggris saja
  #print(string)
  analysis = textblob.TextBlob(string)

  sentiment = ""
  if(analysis.sentiment.polarity==0):
        sentiment = "neutral"
  elif(analysis.sentiment.polarity<0.0000):
        sentiment = "negative"
  elif(analysis.sentiment.polarity>0.0000):
        sentiment = "positive"
  return sentiment


In [6]:
#input keyword and desired number of tweets , prepare csv
keyword = str(input("Please input keyword: "))
number = "10000" #int(input("Number of tweets: "))
date_end  = str(input("end date (yyyy-mm-dd ):"))
date_start  = str(input("start date (yyyy-mm-dd ):"))

c = twint.Config()
c.Until = date_end
c.Since = date_start
c.Search = keyword
#c.Lang = "id" //karna api translate 1000/hari diganti jadi english saja
c.Lang = "en"
c.Pandas_clean = True
c.Debug = True
c.Limit = number
c.Pandas = True
c.Count = True
c.Hide_output = True
c.Filter_retweets = True

twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

sentiment_analysis = [sentimentlysis(x) for x in Tweets_df['tweet']]

perPositive = format(percent(sentiment_analysis.count("positive"), int(Tweets_df.shape[0])),'02f')
perNegative = format(percent(sentiment_analysis.count("negative"), int(Tweets_df.shape[0])),'02f')
perNeutral = format(percent(sentiment_analysis.count("neutral"), int(Tweets_df.shape[0])),'02f')

sizes = [perPositive, perNegative, perNeutral]
labels = ['Positive ['+str(perPositive)+'%] -> '+str(sentiment_analysis.count("positive")) ,'Negative ['+str(perNegative)+'%] -> '+str(sentiment_analysis.count("negative")) ,'Neutral [' +str(perNeutral)+ '%] -> '+str( sentiment_analysis.count("neutral"))]
p, t = plt.pie(sizes, startangle=90)
plt.title("Recent Sentiment Chart \nfor word: "+keyword+" found in "+str(int(Tweets_df.shape[0]))+ " tweets\n")
plt.axis("equal")
plt.legend(p, labels,loc="best")
plt.tight_layout()
plt.show

Please input keyword: avengers endgame
end date (yyyy-mm-dd ):2019-04-24
start date (yyyy-mm-dd ):2019-03-24


CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)


[+] Finished: Successfully collected 10039 Tweets.
